# Guide to use BERT model using Transformers

## Installation

In [1]:
!pip install transformers

## Import modules

In [2]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

2025-01-28 21:42:24.092996: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-28 21:42:24.109557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738089744.127763  263702 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738089744.133187  263702 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-28 21:42:24.168217: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Example-1: Use BERT Tokenizer and BERT pre-trained models

### Load the pre-trained BERT model and its tokenizer.

The model and tokenizer should be specifically designed for question-answering tasks:




In [4]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Tokenization:

Tokenization is the process of converting raw text into a format suitable for input to the model. For BERT, this involves breaking text into tokens and mapping them to their corresponding IDs.

In [5]:
context = """
Blended learning is an educational approach that combines traditional face-to-face classroom methods with online learning activities.
This approach allows students to have greater control over the time, place, path, and pace of their learning. Blended learning 
environments often include interactive online resources, video lectures, and discussion forums, complemented by in-person sessions 
for hands-on activities and group discussions. Studies have shown that blended learning can improve student engagement and outcomes 
by catering to diverse learning styles and providing flexibility. However, its success depends on factors such as access to technology, 
the quality of digital content, and the support provided to students and teachers in using the platform effectively.
"""

In [6]:
question = "What factors influence the success of blended learning?"

In [7]:
# tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

## Model inference

In [8]:
output = model(**inputs)

### Predictions

In [9]:
output

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-6.5302, -5.3346, -8.3935, -8.0089, -8.6484, -8.7349, -8.7004, -8.5287,
         -8.9917, -9.6420, -6.5302, -5.3780, -7.3467, -7.8986, -7.6727, -5.9793,
         -7.3046, -8.1219, -6.6036, -6.2408, -6.0657, -8.1827, -8.2430, -8.4577,
         -7.3873, -6.8120, -7.4977, -8.1131, -4.8053, -6.9933, -6.2131, -6.5300,
         -6.3940, -6.6710, -6.5075, -5.2907, -8.2768, -8.0668, -6.7163, -6.5301,
         -8.3639, -7.7955, -5.3139, -8.3803, -6.4060, -8.4876, -7.0702, -8.4776,
         -8.3069, -6.3323, -8.6986, -8.1035, -6.3115, -7.6198, -5.7240, -7.3479,
         -6.4291, -6.6212, -6.9727, -4.5232, -5.0039, -6.8238, -8.0946, -5.6015,
         -7.4493, -8.4153, -8.1296, -7.1262, -7.0449, -8.5884, -7.4438, -8.9439,
         -8.6388, -6.8865, -8.7398, -8.1713, -7.9134, -8.6095, -6.3948, -8.8774,
         -8.6048, -7.3579, -8.5953, -7.1161, -7.2918, -7.3383, -3.8796, -7.2120,
         -6.1489, -7.4924, -4.1820, -6.5956, -6.0206, -5

In [10]:
answer_start_index = int(torch.argmax(output.start_logits, axis=-1)[0])
answer_end_index = int(torch.argmax(output.end_logits, axis=-1)[0])

- **output.start_logits**: likely represents the output scores or logits generated by the model for the starting position of the answer span in the input passage.
-  **output.end_logits** represents the output scores or logits generated by the model for the ending position of the answer span in the input passage.

The returned response is unique IDs for the token. Using tokenizer to decode every token id into text.

In [11]:
predict = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict)

'access to technology, the quality of digital content, and the support provided to students and teachers in using the platform effectively'

### Important Note: 

The Transformers library ensures compatibility between tokenizers and models. When selecting a model and tokenizer pair, make sure they are compatible and intended for the same architecture. For example, you should use a BERT tokenizer with a BERT model, a GPT-2 tokenizer with a GPT-2 model, and so on.

The Hugging Face model hub provides a variety of pre-trained models and their associated tokenizers. You can find the specific model name in the documentation. For instance, if you're using BERT for sequence classification, you might use bert-base-uncased for both the tokenizer and the model.

Remember to check the specific model's documentation for usage details related to inputs, outputs, and special features, as they can vary based on the model's architecture and intended task. The Transformers library documentation and Hugging Face's GitHub repository are excellent resources to explore more about different models, tokenizers, and their applications.